# Introducing XGA Products

In this tutorial I will briefly touch on XGA's product classes, which are a key part of the internal makeup of the module. They act as an interface between XGA's functions and the data in different types of X-ray data (images, exposure maps, spectra etc.), and contain many useful and convenient methods for analysis and the extraction of information.

In [3]:
from astropy.units import Quantity

from xga.sources import GalaxyCluster

## What are products?

Most XGA products are what we use to wrap various types of data produced by XMM's SAS routines, as well as providing various product specific functionality. Generally, the user is unlikely to ever define a product instance themselves, internal methods in a source object and the SAS wrapper functions will define, check, and store the products. Those products that are generated by XGA's SAS interface have an added ability to parse the stderr output of the SAS routines used to generate them, then flag any recognised errors (by comparing to an archive of known SAS errors).

There are some XGA products that are not wrappers for SAS generated data products, but are purely for storing and providing access to XGA generated data. These are not based on the same superclass as the other products, but are stored in the same way.

Products instantiated by XGA are immediately 'associated' with a source object, though beyond storing the name of the source object in the product's internal structure, they have no knowledge or awareness of the source and its properties. This was by design, to make sure that any functionality built into a product would work regardless of the type of source, and even if it was defined independently of any source at all.

## What types of product are there?

* [BaseProduct](../../xga.sources.rst#xga.products.base.BaseProduct) - The superclass for many of the standard XGA product classes, there isn't really any reason for a BaseProduct to be declared

    * **[EventList](../../xga.sources.rst#xga.products.misc.EventList)** - 

    * **[Image](../../xga.sources.rst#xga.products.phot.Image)** - 

    * **[ExpMap](../../xga.sources.rst#xga.products.phot.ExpMap)** - 

    * **[RateMap](../../xga.sources.rst#xga.products.phot.RateMap)** - 

    * **[PSF](../../xga.sources.rst#xga.products.phot.PSF)** - 
    
    * **[Spectrum](../../xga.sources.rst#xga.products.spec.Spectrum)** - 


* **BaseAggregateProduct** - 
    * **PSFGrid** - 
    
    * **AnnularSpectra** - 



* **BaseProfile1D** - 
    * **SurfaceBrightness1D** -
    
    * **GasDensity1D** - 
    
    * **GasMass1D** - 


* **BaseAggregateProfile1D** - 


* **ScalingRelation** - 


* **AggregateScalingRelation** - 


## Retrieving products from a source